# Accurate Learning with Neural Networks - from Theory to Practice

> Accompanying code for the paper 'Training ReLU Networks to high uniform accuracy is 
> intractable'. Implemented in [PyTorch](https://pytorch.org/), experiment execution and tracking using [Ray Tune](https://www.ray.io/ray-tune) 
> and [Weights & Biases](https://wandb.ai/).

In [ ]:
from IPython.display import display, HTML
from pathlib import Path

from theory2practice import utils, analysis

%load_ext autoreload
%load_ext tensorboard
%autoreload 2

display(HTML("<style>.container { width:100% !important; }</style>"))
spec_dir = Path("specs")
results_dir = Path("results")

## Run Experiments

In [ ]:
project = "test"
experiment = "exp_0"
resume = False

exp_file = spec_dir / project / f"{experiment}.yaml"
runner_file = spec_dir / "runner_resume.yaml" if resume else spec_dir / "runner.yaml"
# !python main.py -e {exp_file} -r {runner_file}

## Tensorboard

In [ ]:
tensorboard --logdir {results_dir / project}

## Visualize Experiments

In [ ]:
# hold ctr to select
project = "test"

selector = analysis.selector(path=results_dir / project)
selector

In [ ]:
visualizer = analysis.Visualizer(exp_dirs=selector.value)
visualizer.initialize()

metrics = list(visualizer.metrics.keys())
print(f"Available metrics: {metrics}")

In [ ]:
# for pdes visualizer needs to be adapted

# visualizer = analysis.Visualizer(
#     exp_dirs=selector.value,
#     update_keys={
#         "n_samples": "algorithm/n_f",
#         "target_fn": "pde",
#     },
#     update_metrics={
#         "L2": analysis.Metric(2, "test/rel_L2/current"),
#     },
# )
# visualizer.initialize()
# 
# metrics = list(visualizer.metrics.keys())
# print(f"Available metrics: {metrics}")

In [ ]:
# select a metric from the list in the output above (defaults to first metric)
metric = metrics[1]

for n_samples, df in visualizer.results[metric]["summary"].items():
    print(f"---- {n_samples} samples ----")
    display(df)

We now plot the min-max rates

$$
    \min_{\text{algo}} \max_{\text{target}} \mathbb{E}\left[ 
    \text{metric}\left(\text{algo}(\text{target})-\text{target}\right)\right],
$$

the theoretical lower bound on the rates (with constant $1$ for simplicity), and the min-avg rates

$$
    \min_{\text{algo}}  \frac{1}{\text{n_targets}} \sum_{\text{target}} \mathbb{E}\left[ 
    \text{metric}\left(\text{algo}(\text{target})-\text{target}\right)\right]
$$

over the sample size.

In [ ]:
# Note that only the projects `1d_5x32` and `3d_5x32` consider multiple sample sizes.
# For the other projects (`test` and `1d_sine`) the exponential fit does not make sense and yields a RankWarning.
figs = visualizer.plot()
figs[0].show()

In [ ]:
# plot of runs contributing to the min_max rate (only if model checkpoints have been saved):
for fig in figs[1:]:
    fig.show()

In [ ]:
# show runs contributing to the min_max rate
visualizer.results[metric]["minmax"]

In [ ]:
# show runs contributing to the min_avg rate
visualizer.results[metric]["minavg"]